# MRI Brain Scans

## Analyzed via a Convolutional Neural Network, and using Transfer Learning relying on MobileNet_V2

The machine learning models presented below were jointly created by:

1. **Brianna Chrisman, Ph.D.** *(BioEngineering/BioInformatics, Stanford 2022)*
2. **Stanislav Jabuka, Ph.D.** *(Mathematics, Michigan State, 2002)*

Two models are presented, namely

-  `model_CNN` - A Convolutional Neural Network built from first principles, and
-  `model_MobileNetV2` - A model created using *Transfer Learning*, starting with the Keras application MoblineNetV2. For more information about the latter, please consult https://keras.io/api/applications/
    
The two models are trained for 30 and 50 epochs respecively, after which their accuracy on training and testing MRI scans is as follows:

-  Accuracy on *training* data: **99.46%** and **99.39%** respectively.
-  Accuracy on *testing* data:  **97.48%** and **95.35%** respectively.

Visual representations of the accuracy of both models are given in terms of their *graphs* (with epochs as the input variable) and their *Confusion Matrices*.

## Preparing the training and testing data

This first cell prepares numpy arrays to contain the training and testing images for both models. The input layer for MobileNetV2 has shape (None, 224, 224, 3), prompting us to convert the provided MRI scans to RGB (3 color channels) images of size 224x224. For the CNN model we transform the input images to grayscale pictures of size 150x150, thus creating arrays of shape (None, 150, 150).  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random

train_path = '/content/drive/MyDrive/archive (3)/Training'
test_path = '/content/drive/MyDrive/archive (3)/Testing'

def y_2binary(n):
    z=np.zeros((4), dtype=int)
    z[n]=1
    return z


### Get a list of subdirectories of the train_path
sub_dirs = os.listdir(train_path)


### We shall map names of the subdirectories to numerical values 0, 1, ..., using a dictionary.
output_dictionary = {}
for i in range(len(sub_dirs)):
    output_dictionary.update({sub_dirs[i]: i})

print('The subdirectories are',sub_dirs,'.\n')


### Create empty lists for training/testing data, since we don't know how big they will be.
x_train_CNN = []
x_test_CNN = []

x_train_MobileNetV2 = []
x_test_MobileNetV2 = []

### The training/testing output data is the same for both models.
y_train = []
y_test = []

### Calculate the number of training samples in each subdirectory, find the total number of training samples.
### Convert images to size (150,150) for CNN model, and to size (224,224,3) for MobilenetV2 model. Convert
### the lists to arrays, and store array in x_train.

for dir in sub_dirs:
    sub_dir_path_train = train_path + '/' + dir
    file_counter_train=0
    for path in os.listdir(sub_dir_path_train):
        x=os.path.join(sub_dir_path_train,path)
        if os.path.isfile(x):
            file_counter_train+=1

            image_CNN = cv2.resize(cv2.imread(x,cv2.IMREAD_GRAYSCALE),(150,150))
            image_MobileNetV2 = cv2.resize(cv2.imread(x),(224,224))

            x_train_CNN.append(image_CNN)
            x_train_MobileNetV2.append(image_MobileNetV2)
            y_train.append(y_2binary(output_dictionary[dir]))

    print('The training directory',dir,'has',file_counter_train,'samples.')
print()

### Calculate the number of testing samples in each subdirectory, find the total number of testing samples.
### Convert images to size (224,224,3), then convert them to an np.array, and store array in x_test.

for dir in sub_dirs:
    sub_dir_path_test = test_path + '/' + dir
    file_counter_test=0
    for path in os.listdir(sub_dir_path_test):
        x=os.path.join(sub_dir_path_test,path)
        if os.path.isfile(x):
            file_counter_test+=1

            image_CNN = cv2.resize(cv2.imread(x,cv2.IMREAD_GRAYSCALE),(150,150))
            image_MobileNetV2 = cv2.resize(cv2.imread(x),(224,224))

            x_test_CNN.append(image_CNN)
            x_test_MobileNetV2.append(image_MobileNetV2)
            y_test.append(y_2binary(output_dictionary[dir]))

    print('The testing directory',dir,'has',file_counter_test,'samples.')
print()

### Convert the now fully populated lists to numpy arrays.
x_train_CNN = np.array(x_train_CNN)/255.0
x_test_CNN = np.array(x_test_CNN)/255.0
x_train_CNN  = x_train_CNN[:,:,:,np.newaxis]
x_test_CNN  =  x_test_CNN[:,:,:,np.newaxis]

x_train_MobileNetV2 = np.array(x_train_MobileNetV2)/255.0
x_test_MobileNetV2 = np.array(x_test_MobileNetV2)/255.0

y_train = np.array(y_train)
y_test = np.array(y_test)


### Print information summary, and a sample training and test image.
print()
print('The shape of x_train_CNN is',x_train_CNN.shape,'.')
print('The shape of x_test_CNN is',x_test_CNN.shape,'.\n')
print('The shape of x_train_MobileNetV2 is',x_train_MobileNetV2.shape,'.')
print('The shape of x_test_MobileNetV2 is',x_test_MobileNetV2.shape,'.\n')
print('The shape of y_train is',y_train.shape,'.')
print('The shape of y_test is',y_test.shape,'.\n')


print('The total number of training samples is',len(y_train),'.')
print('The total number of testing samples is',len(y_test),'.\n')


n1 = random.randint(0,len(y_train))
plt.imshow(x_train_CNN[n1], cmap='gray')
plt.title('Sample Trainig Scan')
plt.show()

print()

n2 = random.randint(0,len(x_test_CNN))
plt.imshow(x_test_CNN[n2], cmap='gray')
plt.title('Sample Testing Scan')
plt.show()

The subdirectories are ['pituitary', 'glioma', 'meningioma', 'notumor'] .

The training directory pituitary has 1457 samples.


## Creating the CNN model

The next cell creates the Convolutional Neural Network model `model_CNN`. It is a sequential model, consisting of 4 `Conv2D` layers, with increasing numbers of kernels, and decreasing kernel sizes. MaxPooling and Dropout layers are inserted after each `Conv2D` layer. The model is completed by flattening the output of the convolutional layers, and adding a final dense layer with 4 neurons. A summary of the model is presented after its creation.

In [ ]:
### Creating the CNN Model

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Dense, InputLayer, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Model, Sequential
from keras import metrics

model_CNN = Sequential()
model_CNN.add(InputLayer(input_shape=(150,150,1)))

model_CNN.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model_CNN.add(MaxPooling2D(pool_size=(2,2)))
model_CNN.add(Dropout(0.25))

model_CNN.add(Conv2D(filters=64, kernel_size=(4,4), activation='relu'))
model_CNN.add(MaxPooling2D(pool_size=(2,2)))
model_CNN.add(Dropout(0.25))

model_CNN.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model_CNN.add(MaxPooling2D(pool_size=(2,2)))
model_CNN.add(Dropout(0.25))

model_CNN.add(Conv2D(filters=256, kernel_size=(2,2), activation='relu'))
model_CNN.add(MaxPooling2D(pool_size=(2,2)))
model_CNN.add(Dropout(0.25))

model_CNN.add(Flatten())
model_CNN.add(Dense(2048, activation='relu'))
model_CNN.add(Dropout(0.25))

model_CNN.add(Dense(4, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
model_CNN.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_CNN.summary()

## Training the CNN model

We train the CNN model for 30 epochs on the prepared training data.

In [ ]:
import time
import math

t1=time.time()
history_CNN = model_CNN.fit(x_train_CNN, y_train, batch_size = 64, epochs = 30)
model_CNN.save('CNN_30epochs')
t2=time.time()
print('The training of 30 epochs of the CNN model took',round((t2-t1)/60),'minutes.')

## Creating the MobileNet_V2 transfer model

The next cell creates the trasnfer model `model_MobileNetV2` by starting with `MobileNet_V2`. We tag on a new output layer - a dense layer with 4 neurons. Only the paramters of the last layer are trainable. A summary of the created model is displayed.

In [ ]:
##### Create the Transfer Learning model from Keras application MobileNetV2.
import sys
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.mobilenet_v2 import MobileNetV2

model_temp = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_temp.trainable = False
x = model_temp.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(4, activation='softmax')(x)

model_MobileNetV2 = Model(inputs=model_temp.input, outputs=x)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
model_MobileNetV2.compile(loss='categorical_crossentropy', optimizer=optimizer,
                     metrics=['accuracy'])

model_MobileNetV2.summary()

## Training the MobilNet_V2 transfer model

Next we train the transfer model for 50 epochs on the prepared training data.

In [ ]:
t3=time.time()
history_MobileNetV2 = model_MobileNetV2.fit(x_train_MobileNetV2, y_train, epochs=50, batch_size=64)
model_MobileNetV2.save('MobileNetV2_50epochs')
t4=time.time()
print()
print('The training of 50 epochs of the Transfer Learning model took',round((t4-t3)/60),'minutes.')

## Accuracy and loss on testing data

We test the accuracy and compute the loss of both models on the provided testing data.

In [ ]:
val_loss_CNN, val_accuracy_CNN = model_CNN.evaluate(x_test_CNN, y_test)
val_loss_MobileNetV2, val_accuracy_MobileNetV2 = model_MobileNetV2.evaluate(x_test_MobileNetV2, y_test)

print()
print('The loss of the CNN model is',round(val_loss_CNN,4),' while its accuracy on testing datais ',round(100*val_accuracy_CNN,2),'%.')
print('The loss of the Transfer Learning model is',round(val_loss_MobileNetV2,4),' while its accuracy on testing datais ',round(100*val_accuracy_MobileNetV2,2),'%.')


## Accuracy and loss plots
The next two cells present graphs of the accuracy and loss, viewed as functions of the epochs.  

In [ ]:
plt.plot(history_CNN.history["loss"],c="red")
plt.plot(history_CNN.history["accuracy"],c="green")
plt.title("Loss and Accuracy in the CNN Model")
plt.ylabel("Loss/Accuracy")
plt.xlabel("Epochs")
plt.legend(['loss','accuracy'])
plt.show()

In [ ]:
plt.plot(history_MobileNetV2.history["loss"],c="red")
plt.plot(history_MobileNetV2.history["accuracy"],c="green")
plt.title("Loss and Accuracy in the Transfer Learning Model")
plt.ylabel("Loss/Accuracy")
plt.xlabel("Epochs")
plt.legend(['loss','accuracy'])
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

predictions_CNN = np.argmax(model_CNN.predict(x_test_CNN),axis=1)
predictions_MobileNetV2 = np.argmax(model_MobileNetV2.predict(x_test_MobileNetV2),axis=1)
test=np.argmax(y_test,axis=1)

## Confusion Matrices

The next two cells give further graphical representation of the accurcy of both models on the testing data, by displaying their **Confusion Matrices**.

In [ ]:
disp_CNN = ConfusionMatrixDisplay(confusion_matrix(test, predictions_CNN))
disp_CNN.plot()
disp_CNN.ax_.set_title("Confusion matrix for the CNN model.")
plt.show()

In [ ]:
disp_MobileNetV2 = ConfusionMatrixDisplay(confusion_matrix(test, predictions_MobileNetV2))
disp_MobileNetV2.plot()
disp_MobileNetV2.ax_.set_title("Confusion matrix for the Transfer Learning model.")
plt.show()